In [1]:
from abc import ABC
import torch.nn.init as init
import torch.nn as nn
import torch

In [2]:
class Net(nn.Module, ABC):
    def __init__(self):
        super(Net, self).__init__()
        self.pool1 = nn.MaxPool2d(2, 2)

    def forward(self, x):
        x = self.pool1(x)
        return x


net = Net()

'''
The hook will be called every time before forward() is invoked. It should have the following signature:
    hook(module, input) -> None or modified input
'''


def forward_pre_hook(module,
                     data_input):  # 模块的输入参数元组(即该模块forward方法的参数构成的参数元组)
    """对输入进行相关操作"""
    print("初始输入为:\n{}".format(data_input))
    data_input[0].mul_(5)  # 初始输入*=5
    return data_input[0].mul_(2)  # 初始输入再*=2


net.pool1.register_forward_pre_hook(forward_pre_hook)  # 模块前向传播之前

fake_img = torch.ones((1, 1, 4, 4))
output = net(fake_img)
print('修改初始输入后,输出为:\n', output)  # 结果为1*5*2=10

初始输入为:
(tensor([[[[1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [1., 1., 1., 1.],
          [1., 1., 1., 1.]]]]),)
修改初始输入后,输出为:
 tensor([[[[10., 10.],
          [10., 10.]]]])


In [3]:
class Net1(nn.Module):
    def __init__(self):
        super(Net1, self).__init__()
        self.conv1 = nn.Conv2d(1, 2, 3)

    def forward(self, x):
        x = self.conv1(x)
        return x


net1 = Net1()
init.constant_(net1.conv1.weight, 1)  # 将conv1的卷积核初始化为全1的tensor
weight_lst = list()


def forward_pre_hook1(module, data_input):
    """对模块权重进行相关操作"""

    # module.weight为可变数据类型,模型运行过程中,module.weight的改变都为in-place操作,若要保存其渐变过程,必须使用.clone()函数先拷贝,得到其副本
    weight = module.weight.clone()
    weight_lst.append(weight)
    print('初始卷积核:\n{}'.format(weight))
    with torch.no_grad():
        module.weight.div_(2)  # 初始卷积核/=2
    return module.weight.data.div_(5)  # 初始卷积核/=5


net1.conv1.register_forward_pre_hook(forward_pre_hook1)

fake_img = torch.ones((1, 1, 4, 4))
out1 = net1(fake_img)
print('修改后的卷积核为:\n', net1.conv1.weight)  # 结果为1/2/5=0.10000

初始卷积核:
tensor([[[[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]]],


        [[[1., 1., 1.],
          [1., 1., 1.],
          [1., 1., 1.]]]], grad_fn=<CloneBackward>)
修改后的卷积核为:
 Parameter containing:
tensor([[[[0.1000, 0.1000, 0.1000],
          [0.1000, 0.1000, 0.1000],
          [0.1000, 0.1000, 0.1000]]],


        [[[0.1000, 0.1000, 0.1000],
          [0.1000, 0.1000, 0.1000],
          [0.1000, 0.1000, 0.1000]]]], requires_grad=True)


In [4]:
weight_lst

[tensor([[[[1., 1., 1.],
           [1., 1., 1.],
           [1., 1., 1.]]],
 
 
         [[[1., 1., 1.],
           [1., 1., 1.],
           [1., 1., 1.]]]], grad_fn=<CloneBackward>)]